In [1]:
import sys
import os

# เพิ่ม path ของโฟลเดอร์หลักเข้าไปใน sys.path
sys.path.append(os.path.abspath(".."))

# ตอนนี้สามารถ import ได้เลย
from main_graph import graph
from state_schema import AgentState


In [2]:
import pandas as pd
from tqdm import tqdm

In [3]:
import sys
import os
import pandas as pd
from tqdm import tqdm
import asyncio
import nest_asyncio

nest_asyncio.apply()  # ⭐️ สำคัญสำหรับ Jupyter / IPython

sys.path.append(os.path.abspath(".."))

# โหลด DataFrame
df = pd.read_excel("question_2categorylevel.xlsx")
df.columns = df.columns.str.strip()

results = []

async def process_query(query, expected_lv1, expected_lv2):
    try:
        initial_state = AgentState(user_query=query)
        state_raw = await graph.ainvoke(initial_state)
        state_obj = AgentState(**state_raw)

        classification = getattr(state_obj, "classification_result", None)
        if classification is None:
            # ถ้าไม่มี classification_result
            classification = {}

        out_of_domain = getattr(classification, "out_of_domain", False)
        out_of_domain_reason = getattr(classification, "out_of_domain_reason", "")

        # ถ้าไม่มีเหตุผล ให้ใส่ default
        if out_of_domain and not out_of_domain_reason:
            out_of_domain_reason = "Out-of-domain without specific reason."

        if out_of_domain:
            predicted_lv1 = "OUT_OF_DOMAIN"
            predicted_lv2 = "OUT_OF_DOMAIN"
            correct_lv1 = expected_lv1.strip().upper() == "OUT_OF_DOMAIN"
            correct_lv2 = expected_lv2.strip().upper() == "OUT_OF_DOMAIN"

            # log กรณี Out-of-domain
            print(f"🔹 Out-of-domain detected: {query}")
            print(f"   Reason: {out_of_domain_reason}")

        else:
            cat_lv1 = getattr(classification, "category_level_1", []) or []
            if cat_lv1 and len(cat_lv1) > 0:
                predicted_lv1 = cat_lv1[0].category
            else:
                predicted_lv1 = ""

            cat_lv2 = getattr(classification, "category_level_2", []) or []
            if cat_lv2 and len(cat_lv2) > 0:
                predicted_lv2 = cat_lv2[0].subcategory
            else:
                predicted_lv2 = ""

            correct_lv1 = predicted_lv1 == expected_lv1
            correct_lv2 = predicted_lv2 == expected_lv2

        latency = state_obj.latency or {}
        total_latency = round(
            sum(v for k, v in latency.items() if isinstance(v, float)), 3
        )

        return {
            "query": query,
            "expected_lv1": expected_lv1,
            "predicted_lv1": predicted_lv1,
            "correct_lv1": correct_lv1,
            "expected_lv2": expected_lv2,
            "predicted_lv2": predicted_lv2,
            "correct_lv2": correct_lv2,
            "out_of_domain": out_of_domain,
            "out_of_domain_reason": out_of_domain_reason,
            "latency_out_of_domain": latency.get("out_of_domain"),
            "latency_clarification": latency.get("clarification"),
            "latency_classification": latency.get("classification"),
            "tokens_out_of_domain": latency.get("tokens_out_of_domain"),
            "tokens_clarification": latency.get("tokens_clarification"),
            "tokens_classification": latency.get("tokens_classification"),
            "total_latency": total_latency,
            "error": ""
        }
    except Exception as e:
        return {
            "query": query,
            "expected_lv1": expected_lv1,
            "predicted_lv1": "",
            "correct_lv1": False,
            "expected_lv2": expected_lv2,
            "predicted_lv2": "",
            "correct_lv2": False,
            "out_of_domain": None,
            "out_of_domain_reason": "",
            "latency_out_of_domain": None,
            "latency_clarification": None,
            "latency_classification": None,
            "tokens_out_of_domain": None,
            "tokens_clarification": None,
            "tokens_classification": None,
            "total_latency": None,
            "error": str(e)
        }

async def main():
    for _, row in tqdm(df.iterrows(), total=len(df)):
        query = row["question"]
        expected_lv1 = row["category_level1"]
        expected_lv2 = row["category_level2"]
        res = await process_query(query, expected_lv1, expected_lv2)
        results.append(res)

    df_results = pd.DataFrame(results)
    df_results.to_csv("evaluation_results.csv", index=False, encoding="utf-8-sig")
    print("✅ Evaluation completed and saved to evaluation_results.csv")

# ⭐️ ใน Jupyter Notebook ให้รันด้วย:
await main()


  0%|          | 0/39 [00:00<?, ?it/s]

START PIPELINE
Step 1: Clarity Check
Clarity Result: clarification_needed=False reason='คำถามชัดเจน มีบริบทและหัตถการครบถ้วน'
should_terminate? False
Step 2: Category Classification


  3%|▎         | 1/39 [00:15<09:55, 15.67s/it]

RESPONSE SET: หมวดหมู่ที่ตรวจพบ:
- หมวดหลัก: หัตถการ
- หมวดย่อย: การใช้ยาชา
START PIPELINE
Step 1: Clarity Check
Clarity Result: clarification_needed=False reason='คำถามชัดเจน มีบริบทและหัตถการครบถ้วน'
should_terminate? False
Step 2: Category Classification


  5%|▌         | 2/39 [00:24<07:16, 11.80s/it]

...

 97%|█████████▋| 38/39 [10:25<00:15, 15.84s/it]

RESPONSE SET: หมวดหมู่ที่ตรวจพบ:
- หมวดหลัก: การปฏิบัติตัวหลังทำหัตถการ
- หมวดย่อย: food
START PIPELINE
Step 1: Clarity Check
Clarity Result: clarification_needed=False reason='มีบริบทและหัตถการครบถ้วน'
should_terminate? False
Step 2: Category Classification


100%|██████████| 39/39 [10:44<00:00, 16.53s/it]

RESPONSE SET: หมวดหมู่ที่ตรวจพบ:
- หมวดหลัก: การปฏิบัติตัวหลังทำหัตถการ
- หมวดย่อย: oral hygiene


PermissionError: [Errno 13] Permission denied: 'evaluation_results.csv'

In [ ]:
results

[{'query': ' ยาชาถอนฟันจะอยู่ได้นานกี่ชั่วโมง?',
  'expected_lv1': 'หัตถการ',
  'predicted_lv1': 'อาการ/ภาวะแทรกซ้อน',
  'correct_lv1': False,
  'expected_lv2': 'การใช้ยาชา',
  'predicted_lv2': 'wound healing',
  'correct_lv2': False,
  'error': ''},
 {'query': 'หลังถอนฟัน อาการผิดปกติแบบไหนที่ควรกลับไปพบทันตแพทย์',
  'expected_lv1': 'อาการ/ภาวะแทรกซ้อน',
  'predicted_lv1': 'อาการ/ภาวะแทรกซ้อน',
  'correct_lv1': True,
  'expected_lv2': 'overall',
  'predicted_lv2': 'wound healing',
  'correct_lv2': False,
  'error': ''},
 {'query': 'ความเสี่ยงและผลกระทบที่เกิดจากการถอนฟัน',
  'expected_lv1': 'อาการ/ภาวะแทรกซ้อน',
  'predicted_lv1': 'อาการ/ภาวะแทรกซ้อน',
  'correct_lv1': True,
  'expected_lv2': 'overall',
  'predicted_lv2': 'wound healing',
  'correct_lv2': False,
  'error': ''},
 {'query': 'สัญญาณของการติดเชื้อหลังถอนฟัน',
  'expected_lv1': 'อาการ/ภาวะแทรกซ้อน',
  'predicted_lv1': 'อาการ/ภาวะแทรกซ้อน',
  'correct_lv1': True,
  'expected_lv2': 'overall',
  'predicted_lv2': 'wound healin

In [4]:
import pandas as pd

# สมมุติว่า results เป็น list ของ dictionary
results_df4 = pd.DataFrame(results)

# แล้วค่อยบันทึกเป็น Excel
results_df4.to_excel("results_category_11_7.xlsx", index=False)



In [5]:
from tqdm import tqdm
import pandas as pd

df = pd.read_excel("question_clari_test.xlsx")
df.columns = df.columns.str.strip()
df["clarification_needed_expected"] = True
results_clari = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    query = row["question"]
    expected_clarification = row["clarification_needed_expected"]

    try:
        initial_state = AgentState(user_query=query)
        state = graph.invoke(initial_state)
        state_obj = AgentState(**state)

        # ✅ ดึงผลลัพธ์จาก state
        predicted_clarification = getattr(state_obj.classification_result, "clarification_needed", None)
        clarification_reason = getattr(state_obj.classification_result, "clarification_reason", "")
        out_of_domain = getattr(state_obj.classification_result, "out_of_domain", None)
        out_of_domain_reason = getattr(state_obj.classification_result, "out_of_domain_reason", "")

        correct_clarification = predicted_clarification == expected_clarification

        latency = state_obj.latency or {}
        total_latency = round(sum(v for k, v in latency.items() if isinstance(v, float)), 3)

        # ✅ เพิ่ม note เฉพาะกรณี out_of_domain=True
        note = ""
        if out_of_domain is True:
            note = f"out_of_domain = True, reason: {out_of_domain_reason}"

        results_clari.append({
            "query": query,
            "expected_clarification": expected_clarification,
            "predicted_clarification": predicted_clarification,
            "correct_clarification": correct_clarification,
            "clarification_reason": clarification_reason,
            "latency_clarification": latency.get("clarification"),
            "tokens_clarification": latency.get("tokens_clarification"),
            "total_latency": total_latency,
            "note": note,
            "error": ""
        })

    except Exception as e:
        results_clari.append({
            "query": query,
            "expected_clarification": expected_clarification,
            "predicted_clarification": "",
            "correct_clarification": False,
            "clarification_reason": "",
            "latency_clarification": None,
            "tokens_clarification": None,
            "total_latency": None,
            "note": "",
            "error": str(e)
        })


  0%|          | 0/15 [00:00<?, ?it/s]

START PIPELINE
Step 1: Clarity Check


  7%|▋         | 1/15 [00:05<01:13,  5.28s/it]

Clarity Result: clarification_needed=True reason='ไม่มีบริบททางทันตกรรม'
should_terminate? True
TERMINATE after Clarity
START PIPELINE
Step 1: Clarity Check


 13%|█▎        | 2/15 [00:18<02:10, 10.05s/it]

...

Clarity Result: clarification_needed=True reason='ไม่มีเจตนาถาม'
should_terminate? True
TERMINATE after Clarity
START PIPELINE
Step 1: Clarity Check


 93%|█████████▎| 14/15 [02:35<00:11, 11.87s/it]

Clarity Result: clarification_needed=True reason='ไม่มีเจตนาถาม'
should_terminate? True
TERMINATE after Clarity
START PIPELINE
Step 1: Clarity Check


100%|██████████| 15/15 [02:42<00:00, 10.83s/it]

Clarity Result: clarification_needed=True reason='ไม่ระบุประเภทของหัตถการ เช่น ถอนฟัน ผ่าฟันคุด ฯลฯ'
should_terminate? True
TERMINATE after Clarity


In [15]:
results_clari

[{'query': 'ต้องพักกี่วันถึงจะหายดี',
  'expected_clarification': True,
  'predicted_clarification': None,
  'correct_clarification': False,
  'clarification_reason': '',
  'latency_clarification': None,
  'tokens_clarification': None,
  'total_latency': 24.04,
  'note': 'out_of_domain = True, reason: ',
  'error': ''},
 {'query': 'ยังมีเลือดซึมอยู่นิดหน่อย',
  'expected_clarification': True,
  'predicted_clarification': True,
  'correct_clarification': True,
  'clarification_reason': '',
  'latency_clarification': 9.536,
  'tokens_clarification': 26.0,
  'total_latency': 74.97,
  'note': '',
  'error': ''},
 {'query': 'กินเผ็ดได้ไหมคะ',
  'expected_clarification': True,
  'predicted_clarification': None,
  'correct_clarification': False,
  'clarification_reason': '',
  'latency_clarification': None,
  'tokens_clarification': None,
  'total_latency': 17.241,
  'note': 'out_of_domain = True, reason: ',
  'error': ''},
 {'query': 'ทำไมยังบวมอยู่',
  'expected_clarification': True,
  'pre

In [6]:
import pandas as pd

# สมมุติว่า results เป็น list ของ dictionary
results_clari = pd.DataFrame(results_clari)

# แล้วค่อยบันทึกเป็น Excel
results_clari.to_excel("results_clari_11_7.xlsx", index=False)

In [8]:
import sys
import os
import pandas as pd
from tqdm import tqdm
import asyncio

# ถ้า AgentState และ graph มาจากไฟล์อื่น ให้ import ให้ถูก
# from your_module import AgentState, graph

sys.path.append(os.path.abspath(".."))

# โหลด DataFrame
df = pd.read_excel("question_outofdomain_test.xlsx")
df.columns = df.columns.str.strip()

results = []

async def process_query(query, expected_lv1, expected_lv2):
    try:
        initial_state = AgentState(user_query=query)
        state_raw = await graph.ainvoke(initial_state)
        state_obj = AgentState(**state_raw)

        classification = getattr(state_obj, "classification_result", None)
        if classification is None:
            classification = {}

        out_of_domain = getattr(classification, "out_of_domain", False)
        out_of_domain_reason = getattr(classification, "out_of_domain_reason", "")

        if out_of_domain and not out_of_domain_reason:
            out_of_domain_reason = "Out-of-domain without specific reason."

        if out_of_domain:
            predicted_lv1 = "OUT_OF_DOMAIN"
            predicted_lv2 = "OUT_OF_DOMAIN"
            correct_lv1 = expected_lv1.strip().upper() == "OUT_OF_DOMAIN"
            correct_lv2 = expected_lv2.strip().upper() == "OUT_OF_DOMAIN"

            print(f"🔹 Out-of-domain detected: {query}")
            print(f"   Reason: {out_of_domain_reason}")

        else:
            cat_lv1 = getattr(classification, "category_level_1", []) or []
            if cat_lv1 and len(cat_lv1) > 0:
                predicted_lv1 = cat_lv1[0].category
            else:
                predicted_lv1 = ""

            cat_lv2 = getattr(classification, "category_level_2", []) or []
            if cat_lv2 and len(cat_lv2) > 0:
                predicted_lv2 = cat_lv2[0].subcategory
            else:
                predicted_lv2 = ""

            correct_lv1 = predicted_lv1 == expected_lv1
            correct_lv2 = predicted_lv2 == expected_lv2

        latency = state_obj.latency or {}
        total_latency = round(
            sum(v for k, v in latency.items() if isinstance(v, float)), 3
        )

        return {
            "query": query,
            "expected_lv1": expected_lv1,
            "predicted_lv1": predicted_lv1,
            "correct_lv1": correct_lv1,
            "expected_lv2": expected_lv2,
            "predicted_lv2": predicted_lv2,
            "correct_lv2": correct_lv2,
            "out_of_domain": out_of_domain,
            "out_of_domain_reason": out_of_domain_reason,
            "latency_out_of_domain": latency.get("out_of_domain"),
            "latency_clarification": latency.get("clarification"),
            "latency_classification": latency.get("classification"),
            "tokens_out_of_domain": latency.get("tokens_out_of_domain"),
            "tokens_clarification": latency.get("tokens_clarification"),
            "tokens_classification": latency.get("tokens_classification"),
            "total_latency": total_latency,
            "error": ""
        }
    except Exception as e:
        return {
            "query": query,
            "expected_lv1": expected_lv1,
            "predicted_lv1": "",
            "correct_lv1": False,
            "expected_lv2": expected_lv2,
            "predicted_lv2": "",
            "correct_lv2": False,
            "out_of_domain": None,
            "out_of_domain_reason": "",
            "latency_out_of_domain": None,
            "latency_clarification": None,
            "latency_classification": None,
            "tokens_out_of_domain": None,
            "tokens_clarification": None,
            "tokens_classification": None,
            "total_latency": None,
            "error": str(e)
        }

async def main():
    for _, row in tqdm(df.iterrows(), total=len(df)):
        query = row["question"]
        expected_lv1 = row["category_level1"]
        expected_lv2 = row["category_level2"]
        res = await process_query(query, expected_lv1, expected_lv2)
        results.append(res)

    df_results = pd.DataFrame(results)
    df_results.to_csv("evaluation_results.csv", index=False, encoding="utf-8-sig")
    print("✅ Evaluation completed and saved to evaluation_results.csv")

if __name__ == "__main__":
    asyncio.run(main())


  0%|          | 0/20 [00:00<?, ?it/s]


KeyError: 'category_level1'

In [10]:
import sys
import os
import pandas as pd
from tqdm import tqdm
import asyncio

# ✨ ถ้า AgentState และ graph อยู่ในโมดูลอื่น ให้แก้ตรงนี้
# from your_module import AgentState, graph

sys.path.append(os.path.abspath(".."))

# โหลด DataFrame
df = pd.read_excel("question_outofdomain_test.xlsx")
df.columns = df.columns.str.strip()

results = []

async def process_query(query, expected_lv1, expected_lv2):
    try:
        initial_state = AgentState(user_query=query)
        state_raw = await graph.ainvoke(initial_state)
        state_obj = AgentState(**state_raw)

        classification = getattr(state_obj, "classification_result", None)
        if classification is None:
            classification = {}

        out_of_domain = getattr(classification, "out_of_domain", False)
        out_of_domain_reason = getattr(classification, "out_of_domain_reason", "")

        if out_of_domain and not out_of_domain_reason:
            out_of_domain_reason = "Out-of-domain without specific reason."

        if out_of_domain:
            predicted_lv1 = "OUT_OF_DOMAIN"
            predicted_lv2 = "OUT_OF_DOMAIN"
            correct_lv1 = expected_lv1.strip().upper() == "OUT_OF_DOMAIN"
            correct_lv2 = expected_lv2.strip().upper() == "OUT_OF_DOMAIN"

            print(f"🔹 Out-of-domain detected: {query}")
            print(f"   Reason: {out_of_domain_reason}")

        else:
            cat_lv1 = getattr(classification, "category_level_1", []) or []
            if cat_lv1 and len(cat_lv1) > 0:
                predicted_lv1 = cat_lv1[0].category
            else:
                predicted_lv1 = ""

            cat_lv2 = getattr(classification, "category_level_2", []) or []
            if cat_lv2 and len(cat_lv2) > 0:
                predicted_lv2 = cat_lv2[0].subcategory
            else:
                predicted_lv2 = ""

            correct_lv1 = predicted_lv1 == expected_lv1
            correct_lv2 = predicted_lv2 == expected_lv2

        latency = state_obj.latency or {}
        total_latency = round(
            sum(v for k, v in latency.items() if isinstance(v, float)), 3
        )

        return {
            "query": query,
            "expected_lv1": expected_lv1,
            "predicted_lv1": predicted_lv1,
            "correct_lv1": correct_lv1,
            "expected_lv2": expected_lv2,
            "predicted_lv2": predicted_lv2,
            "correct_lv2": correct_lv2,
            "out_of_domain": out_of_domain,
            "out_of_domain_reason": out_of_domain_reason,
            "latency_out_of_domain": latency.get("out_of_domain"),
            "latency_clarification": latency.get("clarification"),
            "latency_classification": latency.get("classification"),
            "tokens_out_of_domain": latency.get("tokens_out_of_domain"),
            "tokens_clarification": latency.get("tokens_clarification"),
            "tokens_classification": latency.get("tokens_classification"),
            "total_latency": total_latency,
            "error": ""
        }
    except Exception as e:
        return {
            "query": query,
            "expected_lv1": expected_lv1,
            "predicted_lv1": "",
            "correct_lv1": False,
            "expected_lv2": expected_lv2,
            "predicted_lv2": "",
            "correct_lv2": False,
            "out_of_domain": None,
            "out_of_domain_reason": "",
            "latency_out_of_domain": None,
            "latency_clarification": None,
            "latency_classification": None,
            "tokens_out_of_domain": None,
            "tokens_clarification": None,
            "tokens_classification": None,
            "total_latency": None,
            "error": str(e)
        }

async def main():
    print("Columns in DataFrame:", df.columns.tolist())

    for _, row in tqdm(df.iterrows(), total=len(df)):
        query = row["question"]

        # ตรวจว่ามีเหตุผล out-of-domain หรือไม่
        expected_out_of_domain_reason = str(row["เหตุผลที่อยู่นอกขอบเขต (Out of Domain)"]).strip()
        expected_is_out_of_domain = bool(expected_out_of_domain_reason)

        # ถ้าเป็น out-of-domain ให้ expected lv1/lv2 เป็น OUT_OF_DOMAIN
        expected_lv1 = "OUT_OF_DOMAIN" if expected_is_out_of_domain else ""
        expected_lv2 = "OUT_OF_DOMAIN" if expected_is_out_of_domain else ""

        res = await process_query(query, expected_lv1, expected_lv2)
        results.append(res)

    df_results = pd.DataFrame(results)
    df_results.to_excel("outofdomain_results_11_7.xlsx", index=False, engine="openpyxl")
    print("✅ Evaluation completed and saved to evaluation_results.csv")

if __name__ == "__main__":
    asyncio.run(main())


Columns in DataFrame: ['question', 'เหตุผลที่อยู่นอกขอบเขต (Out of Domain)']


  0%|          | 0/20 [00:00<?, ?it/s]

START PIPELINE
Step 1: Clarity Check


  5%|▌         | 1/20 [00:06<01:55,  6.10s/it]

Clarity Result: clarification_needed=True reason='ไม่มีบริบททางทันตกรรม'
should_terminate? True
TERMINATE after Clarity
START PIPELINE
Step 1: Clarity Check


...

START PIPELINE
Step 1: Clarity Check


 95%|█████████▌| 19/20 [02:47<00:06,  6.41s/it]

Clarity Result: clarification_needed=True reason='ไม่มีเจตนาถาม'
should_terminate? True
TERMINATE after Clarity
START PIPELINE
Step 1: Clarity Check


100%|██████████| 20/20 [02:52<00:00,  8.61s/it]

Clarity Result: clarification_needed=True reason='ไม่มีบริบททางทันตกรรม'
should_terminate? True
TERMINATE after Clarity
✅ Evaluation completed and saved to evaluation_results.csv


In [9]:
import pandas as pd

df_results = pd.DataFrame(results)

# Save เป็น Excel
df_results.to_excel("outofdomain_results_11_7.xlsx", index=False, engine="openpyxl")

print("Evaluation completed and saved to outofdomain_results_11_7.xlsx")


Evaluation completed and saved to outofdomain_results_11_7.xlsx
